In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-$(lsb_release -c -s) main" \
  | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -q update
!sudo apt-get -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  1022  100  1022    0     0  14502      0 --:--:-- --:--:-- --:--:-- 14600
OK
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://packages.cloud.google.com/apt gcsfuse-jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports

In [ ]:
!mkdir /content/gcs_mount
!gcsfuse --implicit-dirs data_storage_bucket1 /content/gcs_mount
%cd gcs_mount
!ls

mkdir: cannot create directory ‘/content/gcs_mount’: File exists
{"timestamp":{"seconds":1760572745,"nanos":726090166},"severity":"INFO","message":"Start gcsfuse/3.4.0 (Go version go1.24.5) for app \"\" using mount point: /content/gcs_mount\n"}
{"timestamp":{"seconds":1760572745,"nanos":726127279},"severity":"INFO","message":"GCSFuse config","config":{"AppName":"","CacheDir":"","CloudProfiler":{"AllocatedHeap":true,"Cpu":true,"Enabled":false,"Goroutines":false,"Heap":true,"Label":"gcsfuse-0.0.0","Mutex":false},"Debug":{"ExitOnInvariantViolation":false,"Fuse":false,"Gcs":false,"LogMutex":false},"DisableAutoconfig":false,"EnableAtomicRenameObject":true,"EnableGoogleLibAuth":false,"EnableHns":true,"EnableNewReader":true,"FileCache":{"CacheFileForRangeRead":false,"DownloadChunkSizeMb":200,"EnableCrc":false,"EnableODirect":false,"EnableParallelDownloads":false,"ExperimentalExcludeRegex":"","ExperimentalParallelDownloadsDefaultOn":true,"MaxParallelDownloads":24,"MaxSizeMb":-1,"ParallelDownlo

In [ ]:

!pip install numpy==1.26.1
!pip install datasets
!pip install libomp-dev
!pip install faiss-gpu-cu12
!pip install chonkie
!pip install sentence_transformers
!pip install duckdb
!pip install openai-agents


In [ ]:
from datasets import load_dataset
import numpy as np
import re
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer



In [ ]:
#device = torch.device("cuda")
model = SentenceTransformer('sentence-transformers/msmarco-roberta-base-v2')
wiki = load_dataset("wikimedia/wikipedia", "20231101.en")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
chunker = SentenceChunker()

#csv_file = "chunks.csv"

def article_csv():
    id = 0
    with open(csv_file, "w") as f:
      summary_list1 = []
      for i, row in enumerate(wiki["train"]):

          text = row["text"].replace("\n", " ")
          summary_list1.append(text)
          if len(summary_list1) >= 4096:
                batched_chunks = chunker.chunk_batch(texts = summary_list1, show_progress=True)
                for j in batched_chunks:
                    # for each fhunk in article j
                    for c in j:
                      f.write(f"{id}\t{c.text}\n")
                      id += 1
                summary_list1 = []
      if summary_list1:

          batched_chunks = chunker.chunk_batch(texts = summary_list1, show_progress=True)
          for k in batched_chunks:
              for c in k:
                f.write(f"{id}\t{c.text}\n")
                id += 1
article_csv()


In [ ]:
import duckdb

#chunker = SentenceChunker()


def article_csv1():

  index = faiss.IndexIDMap(faiss.IndexFlatL2(768))


  con = duckdb.connect("wiki_chunked.duckdb")

  num_rows = int(con.execute("SELECT COUNT(column0) FROM wiki_chunked").fetchone()[0])

  last_index = 0
  last_save = 0
  start_point = 13001280
  stop_point = 14000001
  steps = 8192
  last_row = False

  print(f"There are {num_rows}")

  for i in range(start_point, 13480802, steps):
    print(f"progress {i}")
    last_index = i
    if i + 8192 < num_rows:
      entries = con.execute(f"SELECT * FROM wiki_chunked WHERE column0 >= {i} AND column0 < {i+8192}")
    else:
      entries = con.execute(f"SELECT * FROM wiki_chunked WHERE column0 >= {i} AND column0 < {num_rows}")
      last_row = True
    entries = entries.fetchall()
    print("articles retrieved")
    ids = []
    text = []
    average_size = 0
    for e in entries:
      #print(type(e[1]))
      ids.append(e[0])
      average_size += len(e[1])

      text.append(e[1])
    average_size /= len(entries)
    print("avg size of entries", average_size)
    print("aggregated article lists")
    try:
      #assert(len(ids) == len(text))
      print(len(text))
      print(len(text[0]))

      #tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
      embeddings = model.encode(text, show_progress_bar=True, batch_size=16)
      #with torch.no_grad():
      #  embeddings = model(**tokens).last_hidden_state.mean(dim=1)
      #  embeddings.astype("float32")

      print("embedding complete")
      embeddings = np.ascontiguousarray(embeddings, dtype=np.float32)

      id_array = np.array(np.array(ids),dtype=np.int64)
      index.add_with_ids(embeddings, id_array)
      print("done adding indexes (not individually)")
    except Exception as e:
      for j in range(len(id_array)):
        try:
          embeddings = model.encode(text[j])
          id_array = np.array(np.array(ids[j]),dtype=np.int64)
          embeddings = np.ascontiguousarray(embeddings, dtype=np.float32)
          index.add_with_ids(embeddings, id_array)
        except Exception as e:
          print(f"Bad Chunk at {i}. Skipping...")
    if last_row == True:
      break

  print(f"we finished at {last_index + 8192}")
  faiss.write_index(index, "embeddings_768d_12.bin")

  """
  if last_index < num_rows:
    entries = con.execute(f"SELECT * FROM wiki_chunked WHERE column0 >= {i} AND column0 < {num_rows}")

  for k in range(last_index, num_rows+1):
    try:
      embeddings = model.encode(text[i])
      id_array = np.array(np.array(id[i]),dtype=np.int64)
      index.add_with_ids(embeddings, id_array)
    except Exception as e:
        print(f"Bad Chunk at {i}. Skipping...")

"""


article_csv1()
